# Loading a table of manually normalized data

In [1]:
import pandas as pd

This:

```python
pd.read_csv("aff_norm_update.csv", sep="|")
```

Results in:

`ParserError: Error tokenizing data. C error: Expected 15 fields in line 43757, saw 16`

Some lines in the middle of the data (including a inconsistent line):

In [2]:
!head -n 43757 aff_norm_update.csv | tail -n 3

A `Brasil` value appeared twice. How many rows have a column quantity inconsitency (16 columns instead of 15)?

In [3]:
!sed 's-[^|]--g' aff_norm_update.csv | while read row ; do echo "${#row}" ; done | sort | uniq -c

 506750 14
     37 15


That's just the `|` count. Some of these 16-columns lines have a `|` char quoted inside a field. There are 15 lines that gives us trouble:

In [4]:
!sed 's/".*"//g;/\([^|]*[|]\)\{15\}/!d' aff_norm_update.csv | wc -l

15


In [5]:
!sed 's/".*"//g;/\([^|]*[|]\)\{15\}/!d' aff_norm_update.csv > aff_inconsistent.csv

In [6]:
pd.read_csv("aff_inconsistent.csv", sep="|", header=None, keep_default_na=False).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl
1,S0074-02762013000700921,S0080-62342013000400977,S0080-62341982000100027,S1806-11172015000300309,S0104-530X2015000200391,S0074-02762013000700921,S0080-62342013000400794,S0103-863X2014000300313,S1806-66902015000100099,S1516-14392015000800114,S0074-02762014000100015,S0365-05962013000400523,S2175-78602015000200421,S0074-02762013000700921,S0100-15742015000300680
2,2013,2013,1982,2015,2015,2013,2013,2014,2015,2015,2014,2013,2015,2013,2015
3,rapid-communication,research-article,research-article,research-article,research-article,rapid-communication,research-article,research-article,research-article,research-article,research-article,research-article,research-article,rapid-communication,research-article
4,Memórias do Instituto Oswaldo Cruz,Revista da Escola de Enfermagem da USP,Revista da Escola de Enfermagem da USP,Revista Brasileira de Ensino de Física,Gestão & Produção,Memórias do Instituto Oswaldo Cruz,Revista da Escola de Enfermagem da USP,Paidéia (Ribeirão Preto),Revista Ciência Agronômica,Materials Research,Memórias do Instituto Oswaldo Cruz,Anais Brasileiros de Dermatologia,Rodriguésia,Memórias do Instituto Oswaldo Cruz,Cadernos de Pesquisa
5,v108n7,v47n4,v16n1,v37n3,v22n2,v108n7,v47n4,v24n59,v46n1,v18n suppl 2,v109n1,v88n4,v66n2,v108n7,v45n157
6,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1
7,AFF1,AFF4,AFF2,AFF2,AFF01,AFF2,AFF1,AFF2,AFF02,AFF01,AFF2,AFF03,AFF02,AFF3,AFF1
8,Universidade de São Paulo,Universidade de São Paulo,USP,Universidade Estadual Paulista “Júlio de Mesqu...,Universidade Federal de São Carlos,Fiocruz,FIOCRUZ,Universidade Federal do Rio Grande do Sul,UDESC,Universidade Estadual do Norte Fluminense Darc...,Universidade Federal de Mato Grosso,Federal University of São Paulo,Universidade Estadual de Feira de Santana,Universidade Júlio de Mesquita Filho,Universidade Federal de São Paulo
9,Brasil,Universidade de São Paulo,USP,Universidade Estadual Paulista,Universidade Federal de São Carlos,Brasil,Brasil,Brazil,Departamento de Solos e Recursos Naturais,Universidade Estadual do Norte Fluminense,Brasil,Santa Casa de Misericórdia de São Paulo/OSS,Programa de Pós-graduação em Botânica,Brasil,Unifesp


In [7]:
!head -n 17 aff_norm_update.csv > aff_head.csv

In [8]:
pd.read_csv("aff_head.csv", sep="|", keep_default_na=False).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
coleção,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl
PID,S0104-42302014000200118,S0101-60832016000200020,S1807-59322011000200020,S1516-35982008000500017,S0100-40422008000500040,S0104-07072008000300018,S0037-86822008000600008,S1413-35552008000600005,S0103-166X2009000100009,S1413-73722009000100011,S0103-84782009000600033,S1516-35982010000400025,S1809-29502011000100009,S1983-40632012000200003,S1413-82712012000200011,S1980-00372012000500008
ano de publicação,2014,2016,2011,2008,2008,2008,2008,2008,2009,2009,2009,2010,2011,2012,2012,2012
tipo de documento,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article,research-article
título,Revista da Associação Médica Brasileira,Archives of Clinical Psychiatry (São Paulo),Clinics,Revista Brasileira de Zootecnia,Química Nova,Texto & Contexto - Enfermagem,Revista da Sociedade Brasileira de Medicina Tr...,Brazilian Journal of Physical Therapy,Estudos de Psicologia (Campinas),Psicologia em Estudo,Ciência Rural,Revista Brasileira de Zootecnia,Fisioterapia e Pesquisa,Pesquisa Agropecuária Tropical,Psico-USF,Revista Brasileira de Cineantropometria & Dese...
número,v60n2,v43n2,v66n2,v37n5,v31n5,v17n3,v41n6,v12n6,v26n1,v14n1,v39n6,v39n4,v18n1,v42n2,v17n2,v14n5
normalizado,1,1,1,0,1,0,0,0,0,0,0,1,1,1,1,1
id de afiliação,AFF01,AFF3,A03,A01,A01,A02,A02,A02,A01,A01,A02,A03,A02,A01,A02,A04
instituição original,Aanhanguera University - Uniderp,Anhanguera – Uniderp University,Anhanguera Colleges of the Piracicaba,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional,Anhanguera Educacional
paises original,Brazil,Brazil,Brazil,,Brasil,Brasil,,Brasil,Brasil,,Brasil,,,Brasil,Brasil,Brasil


The `instituição original` field, which probably regards to `<institution content-type="original">`, has an adjacent column in the 15 inconsistent rows. Most of them have the same content. Criterion: join the two fields as the original institution text unless it has the same content of an adjacent column.

In [9]:
!awk -F\| -vOFS=\| '\
  $10 != $11 && $10 != $9 {$9 = "\"" $9; $10 = $10 "\""; NF = 15; print} \
  $10 == $11 || $10 == $9 {for (i=10;i<NF;i++) $i = $(i + 1); NF = 15; print} \
' aff_inconsistent.csv > aff_consistent.csv

In [10]:
pd.read_csv("aff_consistent.csv", sep="|", header=None, keep_default_na=False).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl,scl
1,S0074-02762013000700921,S0080-62342013000400977,S0080-62341982000100027,S1806-11172015000300309,S0104-530X2015000200391,S0074-02762013000700921,S0080-62342013000400794,S0103-863X2014000300313,S1806-66902015000100099,S1516-14392015000800114,S0074-02762014000100015,S0365-05962013000400523,S2175-78602015000200421,S0074-02762013000700921,S0100-15742015000300680
2,2013,2013,1982,2015,2015,2013,2013,2014,2015,2015,2014,2013,2015,2013,2015
3,rapid-communication,research-article,research-article,research-article,research-article,rapid-communication,research-article,research-article,research-article,research-article,research-article,research-article,research-article,rapid-communication,research-article
4,Memórias do Instituto Oswaldo Cruz,Revista da Escola de Enfermagem da USP,Revista da Escola de Enfermagem da USP,Revista Brasileira de Ensino de Física,Gestão & Produção,Memórias do Instituto Oswaldo Cruz,Revista da Escola de Enfermagem da USP,Paidéia (Ribeirão Preto),Revista Ciência Agronômica,Materials Research,Memórias do Instituto Oswaldo Cruz,Anais Brasileiros de Dermatologia,Rodriguésia,Memórias do Instituto Oswaldo Cruz,Cadernos de Pesquisa
5,v108n7,v47n4,v16n1,v37n3,v22n2,v108n7,v47n4,v24n59,v46n1,v18n suppl 2,v109n1,v88n4,v66n2,v108n7,v45n157
6,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1
7,AFF1,AFF4,AFF2,AFF2,AFF01,AFF2,AFF1,AFF2,AFF02,AFF01,AFF2,AFF03,AFF02,AFF3,AFF1
8,Universidade de São Paulo,Universidade de São Paulo,USP,Universidade Estadual Paulista “Júlio de Mesqu...,Universidade Federal de São Carlos,Fiocruz,FIOCRUZ,Universidade Federal do Rio Grande do Sul|Brazil,UDESC|Departamento de Solos e Recursos Naturais,Universidade Estadual do Norte Fluminense Darc...,Universidade Federal de Mato Grosso,Federal University of São Paulo|Santa Casa de ...,Universidade Estadual de Feira de Santana|Prog...,Universidade Júlio de Mesquita Filho,Universidade Federal de São Paulo |Unifesp
9,Brasil,Brasil,Brasil,Brasil,Brasil,Brasil,Brasil,,Brasil,Brazil,Brasil,Brazil,Brasil,Brasil,Brazil


In [11]:
!awk -F\| -vOFS=\| '\
  NF == 15 {print} \
  NF == 16 && ($10 != $11 && $10 != $9) {$9 = "\"" $9; $10 = $10 "\""; NF = 15; print} \
  NF == 16 && ($10 == $11 || $10 == $9) {for (i=10;i<NF;i++) $i = $(i + 1); NF = 15; print} \
' aff_norm_update.csv > aff_n15.csv

In [12]:
ndata = pd.read_csv("aff_n15.csv", sep="|", dtype=str, keep_default_na=False)
ndata

,coleção,PID,ano de publicação,tipo de documento,título,número,normalizado,id de afiliação,instituição original,paises original,instituição normalizada,país normalizado ISO-3661,código de país normalizado ISO-3166,estado normalizado ISO-3166,código de estado normalizado ISO-3166
0,scl,S0104-42302014000200118,2014,research-article,Revista da Associação Médica Brasileira,v60n2,1,AFF01,Aanhanguera University - Uniderp,Brazil,Universidade Anhanguera-Uniderp,Brazil,BR,São Paulo,
1,scl,S0101-60832016000200020,2016,research-article,Archives of Clinical Psychiatry (São Paulo),v43n2,1,AFF3,Anhanguera – Uniderp University,Brazil,Universidade Anhanguera-Uniderp,Brazil,BR,,
2,scl,S1807-59322011000200020,2011,research-article,Clinics,v66n2,1,A03,Anhanguera Colleges of the Piracicaba,Brazil,Faculdade Anhanguera,Brazil,BR,São Paulo,
3,scl,S1516-35982008000500017,2008,research-article,Revista Brasileira de Zootecnia,v37n5,0,A01,Anhanguera Educacional,,Anhanguera Educacional,Brazil,BR,,
4,scl,S0100-40422008000500040,2008,research-article,Química Nova,v31n5,1,A01,Anhanguera Educacional,Brasil,Anhanguera Educacional,Brazil,BR,,
5,scl,S0104-07072008000300018,2008,research-article,Texto & Contexto - Enfermagem,v17n3,0,A02,Anhanguera Educacional,Brasil,Anhanguera Educacional,Brazil,BR,,
6,scl,S0037-86822008000600008,2008,research-article,Revista da Sociedade Brasileira de Medicina Tr...,v41n6,0,A02,Anhanguera Educacional,,Anhanguera Educacional,Brazil,BR,,
7,scl,S1413-35552008000600005,2008,research-article,Brazilian Journal of Physical Therapy,v12n6,0,A02,Anhanguera Educacional,Brasil,Anhanguera Educacional,Brazil,BR,,
8,scl,S0103-166X2009000100009,2009,research-article,Estudos de Psicologia (Campinas),v26n1,0,A01,Anhanguera Educacional,Brasil,Anhanguera Educacional,Brazil,BR,,
9,scl,S1413-73722009000100011,2009,research-article,Psicologia em Estudo,v14n1,0,A01,Anhanguera Educacional,,Anhanguera Educacional,Brazil,BR,,


In [13]:
ndata[ndata != ""].count()

coleção                                  506786
PID                                      506786
ano de publicação                        506786
tipo de documento                        506786
título                                   506786
número                                   506786
normalizado                              506786
id de afiliação                          506548
instituição original                     506785
paises original                          306760
instituição normalizada                  506781
país normalizado ISO-3661                506786
código de país normalizado ISO-3166      506786
estado normalizado ISO-3166              475760
código de estado normalizado ISO-3166        20
dtype: int64

In [14]:
unames_orig = ndata["instituição original"].unique()
unames_norm = ndata["instituição normalizada"].unique()
len(unames_orig), len(unames_norm)

(7304, 271)

# Using DBSCAN for word clustering with scikit-learn

DBSCAN means *Density-based spatial clustering of applications with noise*. [Wikipedia link](https://en.wikipedia.org/wiki/DBSCAN).

The goal is to use it with the Levenshtein distance (and other string/text "metrics"). The `sklearn.cluster.dbscan` requires a 2D array of floats as its input, but it uses the `sklearn.metrics.pairwise.pairwise_distances` to compute their distances, so we can use a single-column of indices instead. 

The imported Levenshtein distance implementation is the [python-Levenshtein from PyPI](https://pypi.org/project/python-Levenshtein/), version 0.12.0.

In [15]:
import collections
import numpy as np
from sklearn import cluster
import Levenshtein as levenshtein

In [16]:
not_so_random_names = [
  "teste",
  "another name",
  "heey",
  "might",
  "prefffix",
  "test",
  "mighty",
  "different",
  "prefix",
  "other name",
  "something completely different",
  "preffix",
  "other namey",
  "suffix",
  "is fix",
  "preffffix",
  "siffix",
  "mee",
]

In [17]:
def column_indices(names):
    return np.mgrid[:len(names)][:, None]

In [18]:
def indexed_levenshtein_metric(names):
    return lambda a, b: levenshtein.distance(names[int(a[0])], names[int(b[0])])

In [19]:
cluster.dbscan(column_indices(not_so_random_names), eps=2,
               metric=indexed_levenshtein_metric(not_so_random_names), min_samples=1)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17]), array([0, 1, 2, 3, 4, 0, 3, 5, 4, 1, 6, 4, 1, 7, 8, 4, 7, 2]))

In [20]:
def clusterize(names, indexed_metric_factory=indexed_levenshtein_metric, **kwargs):
    core_indices, groups_indices = \
        cluster.dbscan(column_indices(names), metric=indexed_metric_factory(names), **kwargs)
    word_clusters = collections.defaultdict(set)
    for idx, gidx in zip(core_indices, groups_indices):
        word_clusters[gidx].add(names[idx])
    return list(word_clusters.values())

In [21]:
clusterize(not_so_random_names, eps=2, min_samples=1)

[{'test', 'teste'},
 {'another name', 'other name', 'other namey'},
 {'heey', 'mee'},
 {'might', 'mighty'},
 {'preffffix', 'prefffix', 'preffix', 'prefix'},
 {'different'},
 {'something completely different'},
 {'siffix', 'suffix'},
 {'is fix'}]

## Applying DBSCAN in the institution fields

DBSCAN with Levenshtein distance seem perfect for this small data example. On the other hand, when clustering the "normalized" institution field text:

In [22]:
clusterize(unames_norm, eps=2, min_samples=1)

[{'Universidade Anhanguera-Uniderp'},
 {'Faculdade Anhanguera'},
 {'Anhanguera Educacional'},
 {'Universidade Anhanguera'},
 {'Centro Universitário Anhanguera'},
 {'Universidade Bandeirante Anhanguera'},
 {'Universidade Iguaçu'},
 {'Universidade Federal do Piauí'},
 {'Centro Universitário de Várzea Grande'},
 {'Universidade de São Paulo'},
 {'Universidade Estadual Paulista Júlio de Mesquita Filho'},
 {'Brasil', 'Brazil'},
 {''},
 {'Clínica Queiroz'},
 {'Universidade Federal de Minas Gerais'},
 {'Fundação Getúlio Vargas'},
 {'Universidade Estadual de Campinas'},
 {'Universidade Federal do Rio de Janeiro'},
 {'Universidade Federal Rural do Rio de Janeiro'},
 {'Universidade Federal de São Carlos'},
 {'Universidade Federal de Santa Catarina'},
 {'Universidade Federal do Rio Grande do Sul'},
 {'Empresa Brasileira de Pesquisa Agropecuária - Soja'},
 {'Empresa Brasileira de Pesquisa Agropecuária - Gado de Leite'},
 {'Universidade Presbiteriana Mackenzie'},
 {'Fundação Oswaldo Cruz'},
 {'Empre

This doesn't seem to work with our data when compacted (each name appearing only once). `Universidade Federal do Ceará`, `Universidade Federal do Paraná` and `Universidade Federal do Pará` aren't the same university. `Brasil`/`Brazil` is correctly grouped together, but these are spurious values (they shouldn't appear in this field).

On a ~7k list of names, it also takes way too long to compute (computation time is quadratic), and it groups together the the small names (acronyms) since there's a "neighboring path" between them:

In [23]:
%time clusterize(unames_orig, eps=2, min_samples=1)

CPU times: user 3min 1s, sys: 84.4 ms, total: 3min 1s
Wall time: 3min 2s


[{'Aanhanguera University - Uniderp'},
 {'Anhanguera – Uniderp University'},
 {'Anhanguera Colleges of the Piracicaba'},
 {'Anhanguera Educacional'},
 {'Anhanguera Educacional Campus de Leme'},
 {'Anhanguera Educacional Campus de Limeira'},
 {'Anhanguera Educacional Campus de Rio Claro'},
 {'Anhanguera Educacional de Brasília'},
 {'Anhanguera Educacional de Pelotas'},
 {'Anhanguera Education Center'},
 {'Anhanguera Faculty of Brasília'},
 {'Anhanguera UNIDERP'},
 {'Anhanguera University'},
 {'Anhanguera University Center'},
 {'Anhanguera University of São Paulo'},
 {'ANHANGUERA-UNIBAN - Bandeirante University of Sao Paulo'},
 {'Anhanguera-Uniban University'},
 {'Anhanguera-Uniderp University'},
 {'Centro Universitário Anhanguera'},
 {'Centro Universitário Anhanguera de Campo Grande'},
 {'Centro Universitário Anhanguera de Santo André'},
 {'Centro Universitário Anhanguera Niterói'},
 {'Centro Universitário de Goiás - Uni-Anhanguera'},
 {'Centro Universitário Uni-Anhanguera',
  'Centro U

## Alternative metric based on Levenshtein

Let's *pre-normalize* with [unidecode](https://pypi.org/project/Unidecode/) v1.0.22 and bring the text to lowercase, getting rid from some "noise":

In [24]:
import re
from unidecode import unidecode

In [25]:
def pre_normalize(name):
    return " ".join(re.sub("[^a-z ]", "", unidecode(name).lower()).split())

In [26]:
pre_normalize(" Secretaria  da EducaçÃo do: EstaDo de São Paulo")

'secretaria da educacao do estado de sao paulo'

In [27]:
def text_metric(a, b):
    na = pre_normalize(a)
    nb = pre_normalize(b)
    na_list = na.split()
    nb_list = nb.split()
    dist = levenshtein.distance(na, nb)
    rseq = levenshtein.seqratio(na_list, nb_list)
    rset = levenshtein.setratio(na_list, nb_list)
    return .8 * dist / max(1, len(na), len(nb)) + .2 - .1 * (rseq + rset)

In [28]:
{pair: text_metric(*pair) for pair in [
  ("Brasil", "Brazil"),
  ("Universidade Federal do Piauí", "universidade fed. d piaui"),
  ("Universidade Estadual Paulista Júlio de Mesquita Filho", "Universidade Estadual Paulista"),
  ("UNIFESP", "UNESP"),
  ("CNPQ", ""),
  ("Somewhere new at Rebouças", "some where at reboucas"),
]}

{('Brasil', 'Brazil'): 0.16666666666666669,
 ('Universidade Federal do Piauí',
  'universidade fed. d piaui'): 0.1745977011494253,
 ('Universidade Estadual Paulista Júlio de Mesquita Filho',
  'Universidade Estadual Paulista'): 0.4355555555555556,
 ('UNIFESP', 'UNESP'): 0.2619047619047619,
 ('CNPQ', ''): 1.0,
 ('Somewhere new at Rebouças', 'some where at reboucas'): 0.2133653846153846}

In [29]:
def alternative_metric_factory(names):
    return lambda a, b: text_metric(names[int(a[0])], names[int(b[0])])

In [30]:
clusterize(unames_norm, indexed_metric_factory=alternative_metric_factory,
           eps=.167, min_samples=1)

[{'Universidade Anhanguera-Uniderp'},
 {'Faculdade Anhanguera'},
 {'Anhanguera Educacional'},
 {'Universidade Anhanguera'},
 {'Centro Universitário Anhanguera'},
 {'Universidade Bandeirante Anhanguera'},
 {'Universidade Iguaçu'},
 {'Universidade Federal da Bahia',
  'Universidade Federal da Bahia ',
  'Universidade Federal da Paraíba',
  'Universidade Federal de Alagoas',
  'Universidade Federal de Alfenas',
  'Universidade Federal de Goiás',
  'Universidade Federal de Lavras',
  'Universidade Federal de Pelotas',
  'Universidade Federal de Viçosa',
  'Universidade Federal do Acre',
  'Universidade Federal do Amazonas',
  'Universidade Federal do Ceará',
  'Universidade Federal do Maranhão',
  'Universidade Federal do Pampa',
  'Universidade Federal do Paraná',
  'Universidade Federal do Pará',
  'Universidade Federal do Piauí'},
 {'Centro Universitário de Várzea Grande'},
 {'Universidade de São Paulo'},
 {'Universidade Estadual Paulista Júlio de Mesquita Filho'},
 {'Brasil', 'Brazil'}

In [31]:
%time clusterize(unames_orig, indexed_metric_factory=alternative_metric_factory, eps=.167, min_samples=1)

CPU times: user 23min 32s, sys: 1.2 s, total: 23min 33s
Wall time: 23min 36s


[{'Aanhanguera University - Uniderp'},
 {'Anhanguera – Uniderp University',
  'Anhanguera-Uniban University',
  'Anhanguera-Uniderp University'},
 {'Anhanguera Colleges of the Piracicaba'},
 {'Anhanguera Educacional'},
 {'Anhanguera Educacional Campus de Leme',
  'Anhanguera Educacional Campus de Limeira'},
 {'Anhanguera Educacional Campus de Rio Claro'},
 {'Anhanguera Educacional de Brasília'},
 {'Anhanguera Educacional de Pelotas'},
 {'Anhanguera Education Center'},
 {'Anhanguera Faculty of Brasília'},
 {'Anhanguera UNIDERP'},
 {'Anhanguera University'},
 {'Anhanguera University Center'},
 {'Anhanguera University of São Paulo'},
 {'ANHANGUERA-UNIBAN - Bandeirante University of Sao Paulo'},
 {'Centro Universitário Anhanguera',
  'Centro Universitário Uni-Anhanguera',
  'Centro Universitário Uni-anhanguera'},
 {'Centro Universitário Anhanguera de Campo Grande',
  'Centro Universitário Anhanguera de Santo André'},
 {'Centro Universitário Anhanguera Niterói'},
 {'Centro Universitário de 

This clustering makes sense, but it's still not what we're looking for.